In [2]:
import datetime as datetime
import pandas as pd
import numpy as np
#import numpy_financial as npf
from itertools import accumulate
import absbox as abs
from absbox import API, SPV, Generic, mkDealsBy, setDealsBy,prodDealsBy, viz, readFlowsByScenarios, readMultiFlowsByScenarios, readFeesCf, readBondsCf, readAccsCf
from absbox.local.analytics import irr, run_yield_table, flow_by_scenario       #npv, 
from absbox.local.util import positionFlow
from lenses import lens
from dateutil.relativedelta import relativedelta  
from tabulate import tabulate
from itertools import accumulate
from pyxirr import xirr
import matplotlib.pyplot as plt  
import matplotlib.dates as mdates  
from mpl_toolkits.mplot3d import axes3d
from more_itertools import transpose,unzip

# localAPI = API("https://absbox.org/api/dev",check=False)
localAPI = API("http://localhost:8081",check=False)


Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.4

In [36]:
# Asset Assumptions
opb = 489_346_107
original_term = 71
seasoning = 2
cutoff_date = "2023-09-03"
apr = 0.214
cdr = 0.05
cpr = 0.097
recovery_rate = 0.3812
recovery_lag = 6

closing_date = datetime.datetime.strptime(cutoff_date, '%Y-%m-%d').date() + relativedelta(months=+seasoning)  
closing_date = closing_date.strftime('%Y-%m-%d')  
maturity_date = datetime.datetime.strptime(cutoff_date, '%Y-%m-%d').date() + relativedelta(months=+original_term)  
maturity_date = maturity_date.strftime('%Y-%m-%d')  
first_pay_date = datetime.datetime.strptime(cutoff_date, '%Y-%m-%d').date() + relativedelta(months=+1)  
first_pay_date = first_pay_date.strftime('%Y-%m-%d')

# Financing Assumptions, ar = advance rate, c = coupon 
ar_wh = 0.0
c_wh = 0.05
wh_unused_fee = 0.001
wh_servicing_fee = 0.03
wh_trustee_fee = 38_400
stepup_date = "2025-07-03"
stepup_increment = 0.005
wh_target_oc = 0.18
wh_target_ltv = 1 - wh_target_oc 
wh_term = 24
wh_capacity = 350_000_000

sec_date = "2023-11-03"
wh_closing_date = datetime.datetime.strptime(sec_date, '%Y-%m-%d').date() + relativedelta(months=-1)
wh_closing_date = wh_closing_date.strftime('%Y-%m-%d')
ar_a = 0.1174
ar_b = 0.3396 
ar_c = 0.1395 
ar_d = 0.1290 
ar_e = 0.1315 
ar_f = 0.0915 
ar_e = 1 - ar_a - ar_b - ar_c - ar_d - ar_e - ar_f
c_a = 0.0601
c_b = 0.0659
c_c = 0.0696
c_d = 0.0720
c_e = 0.0800
c_f = 0.1046
sec_target_oc_original = 0.015
sec_target_oc_current = 0.095
sec_target_oc_early_amort = 0.14
sec_target_ltv_original = 1 - sec_target_oc_original
sec_target_ltv_current = 1 - sec_target_oc_current
sec_target_ltv_early_amort = 1 - sec_target_oc_early_amort
sec_oc_floor = sec_target_oc_original
sec_fee = 0
sec_reserve_pct = 0.01 
# sec_amt = eop_balance for the sec month 
sec_optional_call = 0.2 

# Deal Assumptions
purchase_cost = 1.0145
purchase_price = opb * purchase_cost
extra_deal_fee = 50_000
total_initial_investment = purchase_price + extra_deal_fee
cnl_trigger = 0.2

In [69]:
ar_e

0.05150000000000002

In [99]:
# Model
mortgage1 = ["Mortgage"
            ,{"originBalance": opb,"originRate": ["fix", apr],"originTerm": original_term,"freq": "monthly","type": "level","originDate": cutoff_date}
            ,{"currentBalance": opb,"currentRate": apr,"remainTerm": original_term - seasoning,"status": "current"}]
revolvingPool = (["constant",mortgage1],("Pool",("Mortgage",{"CDR":cdr},{"CPR":cpr},{"Rate":recovery_rate,"Lag":recovery_lag},None),None,None))
perf = ("Pool",("Mortgage",{"CDR":cdr},{"CPR":cpr},{"Rate":recovery_rate,"Lag":recovery_lag},None),None,None)

ABS = Generic(
    "TEST01"
    ,{"cutoff":cutoff_date,"closing":closing_date,"firstPay":first_pay_date,"payFreq":["DayOfMonth",3],"poolFreq":"MonthEnd","stated":maturity_date}
    ,{'assets':[mortgage1]}
    ,(("acc01",{"balance":0}),("acc02",{"balance":0}),("reserve_acc",{"balance":0, "type":("target",("poolBalance",),sec_reserve_pct)}))
    ,(("WH",{"balance":opb * ar_wh,"rate":c_wh,"originBalance":opb * ar_wh,"originRate":c_wh,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":c_wh},"stepUp":("ladder",stepup_date,stepup_increment,"MonthEnd"),"bondType":{"Sequential":None}})
      ,("A",{"balance":opb * ar_a,"rate":c_a,"originBalance":opb * ar_a,"originRate":c_a,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":c_a},"bondType":{"Sequential":None}})
      ,("B",{"balance":opb * ar_b,"rate":c_b,"originBalance":opb * ar_b,"originRate":c_b,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":c_b},"bondType":{"Sequential":None}})
      ,("C",{"balance":opb * ar_c,"rate":c_c,"originBalance":opb * ar_c,"originRate":c_c,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":c_c},"bondType":{"Sequential":None}})
      ,("D",{"balance":opb * ar_d,"rate":c_d,"originBalance":opb * ar_d,"originRate":c_d,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":c_d},"bondType":{"Sequential":None}})
      ,("E",{"balance":opb * ar_e,"rate":c_e,"originBalance":opb * ar_e,"originRate":c_e,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":c_e},"bondType":{"Sequential":None}})
      ,("F",{"balance":opb * ar_f,"rate":c_f,"originBalance":opb * ar_f,"originRate":c_f,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":c_f},"bondType":{"Sequential":None}})
      ,("G",{"balance":opb * ar_e,"rate":0.0,"originBalance":opb * ar_e,"originRate":0.0,"startDate":sec_date,"maturityDate":maturity_date,"rateType":{"Fixed":0.0},"bondType":{"Equity":None}}))
    ,(("serviceFee",{"type":{"annualPctFee":[("poolBalance",),wh_servicing_fee]},"feeStart":closing_date}),                             # PROBLEM, poolBalance seems to always be 0
      ("trusteeFee", {"type":{"recurFee":["MonthFirst",wh_trustee_fee/12],"feeStart":closing_date,"feeLastPaidDate":maturity_date}}),
      ("unusedFee",{"type":{"annualPctFee":[("poolBalance",),wh_unused_fee]},"feeStart":closing_date}),                                   # PROBLEM
      ("reserveFee",{"type":{"annualPctFee":[("poolBalance",),sec_reserve_pct]},"feeStart":closing_date}),
      ("sec_fee",{"type":{"fixFee":sec_fee,"feeStart":sec_date},"feeStart":closing_date}))
    ,{"revolving":[["payFee","acc01",['serviceFee','trusteeFee','unusedFee']]                         # warehouse phase
         ,["accrueAndPayInt","acc01",["WH"]]
         ,["payFee","reserve_acc",['reserveFee']]
        #  ,["payPrin","acc01",["WH"]]
        #  ,["payPrinResidual","acc01",["WH"]]
        #  ,['payIntResidual',"acc01","WH"]
        #  ,["If",["=",wh_closing_date],["sellAsset",["Current|Defaulted",1.0,0],"acc01"]]
         ]
        ,"amortizing":[
          ["calcFee",'serviceFee','trusteeFee','sec_fee']                       # securitization phase
         ,["payFeeBySeq","acc01",['serviceFee','trusteeFee','sec_fee']]  
         ,["accrueAndPayInt","acc01",["A","B","C","D","E","F"]]
        #  ['IfElse', [('isPaidOff', 'A1'), True], [['transfer', 'reserveAcc', 'acc01']], [['transfer', 'acc01', 'reserveAcc', {'reserve': 'gap'}]]],
        #  ,["If",["=","2022-08-20"],["buyAsset",["Current|Defaulted",1.0,0],"acc01"]]
         ,["payPrin","acc01",["A","B","C","D","E","F"]]
         #,["payPrinResidual","acc01",["A","B","C","D","E","F"]]
         ,['payPrin',"acc01",["G"]]
         ,['payIntResidual',"acc01","G"]
        #  ,["If",["=",maturity_date],["sellAsset",["Current|Defaulted",1.0,0],"acc02"]]
         ]}
    ,[["CollectedInterest","acc01"],["CollectedPrincipal","acc01"],["CollectedPrepayment","acc01"],["CollectedRecoveries","acc01"]]
    ,None,None,None,
    {"BeforeDistribution":{"dealStatusChange":{"condition":[">=",sec_date],"effects":("newStatus","Amortizing"),"status":False,"curable":False}}}
    ,("PreClosing","Revolving"))

r = localAPI.run(ABS,runAssump=[("revolving", *revolvingPool),("call",{"and":[{"afterDate":sec_date},{"poolFactor":sec_optional_call}]})],poolAssump=perf,read=True)

Warning Message from server:
No waterfall distribution found on date2023-10-03with waterfall key DefaultDistribution

In [100]:
readAccsCf(r['agg_accounts'])

Account            acc01                             reserve_acc         \
Field      begin balance       change  end balance begin balance change   
date                                                                      
2023-11-03          0.00         0.00         0.00           0.0    0.0   
2023-11-30          0.00         0.00         0.00           NaN    NaN   
2023-12-03          0.00         0.00         0.00           NaN    NaN   
2023-12-31          0.00  16298309.89  16298309.89           NaN    NaN   
2024-01-03   16298309.89 -16298309.89         0.00           NaN    NaN   
...                  ...          ...          ...           ...    ...   
2028-02-03    7720753.57  -7720753.57         0.00           NaN    NaN   
2028-02-29          0.00   7581936.01   7581936.01           NaN    NaN   
2028-03-03    7581936.01  -7581936.01         0.00           NaN    NaN   
2028-03-31          0.00   7387984.98   7387984.98           NaN    NaN   
2028-04-03    7387984.98  -7387984.98         0.00           NaN    NaN   

Account                 
Field      end balance  
date                    
2023-11-03         0.0  
2023-11-30         NaN  
2023-12-03         NaN  
2023-12-31         NaN  
2024-01-03         NaN  
...                ...  
2028-02-03         NaN  
2028-02-29         NaN  
2028-03-03         NaN  
2028-03-31         NaN  
2028-04-03         NaN  

[107 rows x 6 columns]

In [101]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2023-11-03,4.893461e+08,0.00,0.00,0.00,0.00,0.00,0.00,0.214,None,None,0.000000e+00,0.000000e+00,0,0.00,0.00,0.00
2023-11-30,4.893461e+08,0.00,0.00,0.00,0.00,0.00,0.00,0.214,None,None,0.000000e+00,0.000000e+00,0,0.00,0.00,0.00
2023-12-31,4.796063e+08,3611656.05,8617226.49,4069427.35,2058685.35,0.00,0.00,0.214,None,None,3.611656e+06,4.069427e+06,0,2058685.35,0.00,0.00
2024-01-31,4.697728e+08,3628502.59,8442165.26,4120224.39,2084820.63,0.00,0.00,0.214,None,None,7.240159e+06,8.189652e+06,0,4143505.98,0.00,0.00
2024-02-29,4.600495e+08,3645427.72,8269072.39,4035745.89,2042074.77,0.00,0.00,0.214,None,None,1.088559e+07,1.222540e+07,0,6185580.75,0.00,0.00
2024-03-31,4.508137e+08,3665510.04,8104727.32,3699305.16,1871051.22,0.00,0.00,0.214,None,None,1.455110e+07,1.592470e+07,0,8056631.97,0.00,0.00
2024-04-30,4.412985e+08,3682607.79,7935348.74,3872871.05,1959660.61,0.00,0.00,0.214,None,None,1.823370e+07,1.979757e+07,0,10016292.58,0.00,0.00
2024-05-31,4.320708e+08,3701339.78,7771124.33,3669861.29,1856548.60,0.00,0.00,0.214,None,None,2.193504e+07,2.346744e+07,0,11872841.18,0.00,0.00
2024-06-30,4.227621e+08,3718604.66,7605431.15,3711853.77,1878186.37,784770.85,1273914.49,0.214,None,None,2.565365e+07,2.717929e+07,0,13751027.55,784770.85,1273914.49


In [102]:
r['result']['status']

,Date,From,To
0,2023-11-03,PreClosing,Revolving
1,2023-11-03,Revolving,Amortizing
2,2028-04-03,Amortizing,Called
3,2028-04-03,DealEnd,Clean Up


In [103]:
import pyxirr

In [104]:
closingDate = datetime.datetime.strptime('2023-11-03', '%Y-%m-%d').date()
closingDate

datetime.date(2023, 11, 3)

In [105]:
pyxirr.xirr([closingDate]+r['bonds']["G"].index.to_list(),[-25201324.51]+r['bonds']["G"].cash.to_list())

0.43194230204488293

In [106]:
readBondsCf(r['bonds'])

Bond                  A                                             \
Field           balance   interest   principal    rate        cash   
date                                                                 
2023-12-03  57449232.96       0.00        0.00  0.0601        0.00   
2024-01-03  56232751.44  577026.39  1216481.52  0.0601  1793507.91   
2024-02-03  54535391.48  287033.53  1697359.96  0.0601  1984393.49   
2024-03-03  52825376.82  260410.22  1710014.66  0.0601  1970424.88   
2024-04-03  51198741.62  269640.98  1626635.20  0.0601  1896276.18   
2024-05-03  49541431.85  252907.75  1657309.77  0.0601  1910217.52   
2024-06-03  47933244.84  252878.47  1608187.01  0.0601  1861065.48   
2024-07-03  46206528.91  236777.09  1726715.93  0.0601  1963493.02   
2024-08-03  44524957.53  235855.84  1681571.38  0.0601  1917427.22   
2024-09-03  42846904.28  227272.46  1678053.25  0.0601  1905325.71   
2024-10-03  41181623.58  211651.96  1665280.70  0.0601  1876932.66   
2024-11-03  39555348.47  210206.80  1626275.11  0.0601  1836481.91   
2024-12-03  37924012.07  195392.58  1631336.40  0.0601  1826728.98   
2025-01-03  36333213.37  193578.70  1590798.70  0.0601  1784377.40   
2025-02-03  34749459.83  185458.65  1583753.54  0.0601  1769212.19   
2025-03-03  33148038.08  160209.29  1601421.75  0.0601  1761631.04   
2025-04-03  31634073.39  169200.30  1513964.69  0.0601  1683164.99   
2025-05-03  30090937.85  156263.65  1543135.54  0.0601  1699399.19   
2025-06-03  28584017.79  153595.68  1506920.06  0.0601  1660515.74   
2025-07-03  27075045.92  141197.21  1508971.87  0.0601  1650169.08   
2025-08-03  25600619.83  138201.41  1474426.09  0.0601  1612627.50   
2025-09-03  24130844.98  130675.38  1469774.85  0.0601  1600450.23   
2025-10-03  22676945.41  119199.76  1453899.57  0.0601  1573099.33   
2025-11-03  21250035.19  115751.82  1426910.22  0.0601  1542662.04   
2025-12-03  19823109.71  104969.35  1426925.48  0.0601  1531894.83   
2026-01-03  18426712.42  101184.75  1396397.29  0.0601  1497582.04   
2026-02-03  17037728.18   94057.00  1388984.24  0.0601  1483041.24   
2026-03-03  15642929.88   78550.92  1394798.30  0.0601  1473349.22   
2026-04-03  14309273.39   79847.51  1333656.49  0.0601  1413504.00   
2026-05-03  12958335.44   70683.89  1350937.95  0.0601  1421621.84   
2026-06-03  11634150.72   66144.31  1324184.72  0.0601  1390329.03   
2026-07-03  10312646.49   57469.51  1321504.23  0.0601  1378973.74   
2026-08-03   9016436.45   52639.70  1296210.04  0.0601  1348849.74   
2026-09-03   7725964.25   46023.34  1290472.20  0.0601  1336495.54   
2026-10-03   6451246.32   38164.14  1274717.93  0.0601  1312882.07   
2026-11-03   5195949.67   32929.63  1255296.65  0.0601  1288226.28   
2026-12-03   3945162.00   25666.56  1250787.67  0.0601  1276454.23   
2027-01-03   2716155.32   20137.62  1229006.68  0.0601  1249144.30   
2027-02-03   1494975.71   13864.29  1221179.61  0.0601  1235043.90   
2027-03-03    278482.71    6892.45  1216493.00  0.0601  1223385.45   
2027-04-03         0.00    1421.48   278482.71  0.0601   279904.19   
2027-05-03          NaN        NaN         NaN     NaN         NaN   
2027-06-03          NaN        NaN         NaN     NaN         NaN   
2027-07-03          NaN        NaN         NaN     NaN         NaN   
2027-08-03          NaN        NaN         NaN     NaN         NaN   
2027-09-03          NaN        NaN         NaN     NaN         NaN   
2027-10-03          NaN        NaN         NaN     NaN         NaN   
2027-11-03          NaN        NaN         NaN     NaN         NaN   
2027-12-03          NaN        NaN         NaN     NaN         NaN   
2028-01-03          NaN        NaN         NaN     NaN         NaN   
2028-02-03          NaN        NaN         NaN     NaN         NaN   
2028-03-03          NaN        NaN         NaN     NaN         NaN   
2028-04-03          NaN        NaN         NaN     NaN         NaN   

Bond                   B                                              ...  \
Fi

In [107]:
readBondsCf(r['bonds']).xs("balance",axis=1,level=1).head()

Bond,A,B,C,D,E,F,G,WH
date,,,,,,,,
2023-12-03,57449232.96,1.661819e+08,68263781.92,63125647.80,25201324.51,44775168.79,25201324.51,NaN
2024-01-03,56232751.44,1.626631e+08,66818303.48,61788968.81,24667689.10,43827059.28,25201324.51,NaN
2024-02-03,54535391.48,1.577531e+08,64801423.50,59923897.00,23923106.17,42504159.51,25201324.51,NaN
2024-03-03,52825376.82,1.528066e+08,62769506.62,58044920.10,23172971.98,41171396.83,25201324.51,NaN
2024-04-03,51198741.62,1.481013e+08,60836665.01,56257561.19,22459413.97,39903619.00,25201324.51,NaN


In [108]:
readFeesCf(r['fees'])

Fee        reserveFee         sec_fee          serviceFee              \
Field         balance payment balance payment     balance     payment   
date                                                                    
2023-11-03        0.0     0.0     NaN     NaN        0.00        0.00   
2023-12-03        NaN     NaN     0.0     0.0  1085946.15        0.00   
2024-01-03        NaN     NaN     0.0     0.0        0.00  2212112.53   
2024-02-03        NaN     NaN     0.0     0.0        0.00  1103751.57   
2024-03-03        NaN     NaN     0.0     0.0        0.00  1003898.01   
2024-04-03        NaN     NaN     0.0     0.0        0.00  1058744.15   
2024-05-03        NaN     NaN     0.0     0.0        0.00  1000435.82   
2024-06-03        NaN     NaN     0.0     0.0        0.00  1015591.15   
2024-07-03        NaN     NaN     0.0     0.0        0.00   958842.01   
2024-08-03        NaN     NaN     0.0     0.0        0.00   972932.05   
2024-09-03        NaN     NaN     0.0     0.0        0.00   952146.54   
2024-10-03        NaN     NaN     0.0     0.0        0.00   897929.68   
2024-11-03        NaN     NaN     0.0     0.0        0.00   910457.00   
2024-12-03        NaN     NaN     0.0     0.0        0.00   858531.81   
2025-01-03        NaN     NaN     0.0     0.0        0.00   870043.68   
2025-02-03        NaN     NaN     0.0     0.0        0.00   850336.77   
2025-03-03        NaN     NaN     0.0     0.0        0.00   741502.58   
2025-04-03        NaN     NaN     0.0     0.0        0.00   810844.58   
2025-05-03        NaN     NaN     0.0     0.0        0.00   764116.25   
2025-06-03        NaN     NaN     0.0     0.0        0.00   773176.56   
2025-07-03        NaN     NaN     0.0     0.0        0.00   727515.12   
2025-08-03        NaN     NaN     0.0     0.0        0.00   735623.04   
2025-09-03        NaN     NaN     0.0     0.0        0.00   717288.59   
2025-10-03        NaN     NaN     0.0     0.0        0.00   673885.01   
2025-11-03        NaN     NaN     0.0     0.0        0.00   680594.72   
2025-12-03        NaN     NaN     0.0     0.0        0.00   639145.21   
2026-01-03        NaN     NaN     0.0     0.0        0.00   644944.36   
2026-02-03        NaN     NaN     0.0     0.0        0.00   627522.89   
2026-03-03        NaN     NaN     0.0     0.0        0.00   544656.92   
2026-04-03        NaN     NaN     0.0     0.0        0.00   592691.32   
2026-05-03        NaN     NaN     0.0     0.0        0.00   555691.63   
2026-06-03        NaN     NaN     0.0     0.0        0.00   559284.73   
2026-07-03        NaN     NaN     0.0     0.0        0.00   523317.77   
2026-08-03        NaN     NaN     0.0     0.0        0.00   526053.01   
2026-09-03        NaN     NaN     0.0     0.0        0.00   509790.70   
2026-10-03        NaN     NaN     0.0     0.0        0.00   475851.37   
2026-11-03        NaN     NaN     0.0     0.0        0.00   477326.03   
2026-12-03        NaN     NaN     0.0     0.0        0.00   445050.45   
2027-01-03        NaN     NaN     0.0     0.0        0.00   445702.14   
2027-02-03        NaN     NaN     0.0     0.0        0.00   430209.99   
2027-03-03        NaN     NaN     0.0     0.0        0.00   370256.44   
2027-04-03        NaN     NaN     0.0     0.0        0.00   399318.82   
2027-05-03        NaN     NaN     0.0     0.0        0.00   370851.70   
2027-06-03        NaN     NaN     0.0     0.0        0.00   369501.72   
2027-07-03        NaN     NaN     0.0     0.0        0.00   342045.13   
2027-08-03        NaN     NaN     0.0     0.0        0.00   339915.92   
2027-09-03        NaN     NaN     0.0     0.0        0.00   325398.85   
2027-10-03        NaN     NaN     0.0     0.0        0.00   299777.89   
2027-11-03        NaN     NaN     0.0     0.0        0.00   296502.45   
2027-12-03        NaN     NaN     0.0     0.0        0.00   272295.42   
2028-01-03        NaN     NaN     0.0     0.0        0.00   268269.62   
2028-02-03        NaN     NaN     0.0     0.0        0.00 